依赖项 | 版本--- | ---
SatPy | 0.9.0
# Cartopy 图表
此示例加载多个VIIRS SDR波段I04数据粒度，将其重新采样到已知的`AreaDefinition`，并使用matplotlib在地图上绘制数据。尽管没有直接导入cartopy，但我们使用pyresample的`to_cartopy_crs()`方法创建cartopy CRS对象，并在绘图中使用该对象。
此示例应作为使用SatPy处理数据绘图的起点。更多详细信息可以在[XArray的绘图文档](http://xarray.pydata.org/en/stable/plotting.html#maps)以及[Cartopy的文档](https://scitools.org.uk/cartopy/docs/latest/gallery/geostationary.html#sphx-glr-gallery-geostationary-py)中找到，以帮助构建更复杂的图表。

## 加载数据
使用 ``glob`` 模块可以轻松获取一系列数据文件。创建 Scene 对象后，我们加载 I04 波段数据。

In [1]:
from satpy import Scene
from glob import glob
import matplotlib.pyplot as plt

filenames = glob('/data/data/viirs/conus_day/*t180*.h5')
scn = Scene(reader='viirs_sdr', filenames=filenames)
scn.load(['I04'])

## 重采样到均匀区域
接下来，我们将VIIRS数据放置在高分辨率均匀网格上。我们也可以使用其他任何区域定义。对于极轨卫星数据，由于需要使用的经度/纬度数组的大小，最好先对数据进行重采样，然后再将其绘制到图表上。对于通常已经网格化的静止卫星数据，可以在下面的绘图步骤中直接使用源数据的区域。

In [2]:
my_area = scn['I04'].attrs['area'].compute_optimal_bb_area({'proj': 'lcc', 'lon_0': -95., 'lat_0': 25., 'lat_1': 25., 'lat_2': 25.})
new_scn = scn.resample(my_area)

/Users/davidh/anaconda/envs/polar2grid_py36/lib/python3.6/site-packages/dask/local.py:271: RuntimeWarning: invalid value encountered in greater_equal
  return func(*args2)
/Users/davidh/anaconda/envs/polar2grid_py36/lib/python3.6/site-packages/dask/local.py:271: RuntimeWarning: invalid value encountered in less_equal
  return func(*args2)


## 使用 matplotlib 和 cartopy 绘制数据
要使用cartopy绘制数据，我们首先将pyresample的AreaDefinition转换为cartopy的CRS，然后使用该CRS来创建matplotlib坐标轴。

In [3]:
crs = new_scn['I04'].attrs['area'].to_cartopy_crs()
ax = plt.axes(projection=crs)

ax.coastlines()
ax.gridlines()
ax.set_global()
plt.imshow(new_scn['I04'], transform=crs, extent=crs.bounds, origin='upper')
cbar = plt.colorbar()
cbar.set_label("Kelvin")
plt.show()

/Users/davidh/repos/git/pyresample/pyresample/_cartopy.py:36: UserWarning: 'cartopy' >= 0.17 required for better 'from_proj' functionality.
  warnings.warn("'cartopy' >= 0.17 required for better 'from_proj' "


![标题](Cartopy 图表 1.png)